In [1]:
from mpmath import mp, mpf, findroot, bessely
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

In [2]:
from sage.all import *

R = LaurentSeriesRing(RealField(300), 'x',default_prec=300)
x = R.gen()
N = 16  # Number of terms (adjust as needed)
gamma = RealField(300)(euler_gamma)
d2 = RealField(300)(2)
pi = RealField(300).pi()

# Define J0(x) Taylor expansion at x = 0
def j_series(n, x):
    return sum([(-1)**m * (x/2)**(ZZ(n) + ZZ(2)*ZZ(m)) / (ZZ(m).factorial() * (ZZ(m) + ZZ(n)).factorial()) for m in range(N)])

J0_series = j_series(0, x)

def z_series(x):
    return sum([(-1)**m * (x/2)**(ZZ(2)*ZZ(m)) / ZZ(m).factorial()**ZZ(2) * sum(RealField(300)(1)/RealField(300)(k) for k in range(1, m+1)) for m in range(1, N)])

W0 = (d2/pi) * J0_series
Z0 = -gamma * (d2/pi) * J0_series + RealField(300)(2).log() * (d2/pi) * J0_series + (d2/pi) * z_series(x)

# print("W0(x) =", W0)
# print("Z0(x) =", Z0)

terms = 30

from mpmath import mp, bessely, taylor

def Y0_approx(val):
    # Substitute numeric value and convert to high precision real
    w = W0.truncate(terms)(val)
    z = Z0.truncate(terms)(val)
    return RealField(107)(w) * RealField(107)(val).log() - RealField(107)(z)

mp.prec = 120

print("W = ", RealField(64)(W0.truncate(terms)(0.89357696627916749)))
print("Z = ", RealField(64)(Z0.truncate(terms)(0.89357696627916749)))
print("LOG = ", RealField(110)(0.89357696627916749).log())
print_double_double("", RealField(110)(2).log())
print("r = ", Y0_approx(RealField(100)(0.89357696627916749)))
print("t = ", bessely(0, 0.89357696627916749))

def print_series(z, N):
    for i in range(N):
        coeff = z[i]
        if coeff != 0:
            # print(double_to_hex(RealField(300)(coeff)) + ",")
            print_double_double("", RealField(300)(coeff))

print_series(Z0, 30)

# W0_series = W0.series(x, 2*N)
# Z0_series_expanded = Z0.series(x, 2*N)

# print("W0(x) power series:")
# for i in range(0, 2*N):
#     coeff = W0_series.coefficient(x, i)
#     if coeff != 0:
#         print(f"x^{i}: {coeff}")

W =  0.515740866931212572
Z =  -0.0580326104859750086
LOG =  -0.11252280788079415479644853102767
(0x3c7abc9e3b39803f, 0x3fe62e42fefa39ef),
r =  -2.338927928406201408822986214495e-17
t =  -2.338927928406210311869215576136051e-17
(0xbc5ddfd831a70821, 0x3fb2e4d699cbd01f),
(0xbc6d93e63489aea6, 0xbfc6bbcb41034286),
(0xbc1b88525c2e130b, 0x3f9075b1bbf41364),
(0x3be097334e26e578, 0xbf41a6206b7b973d),
(0x3b51c64a34c78cda, 0x3ee3e99794203bbd),
(0xbb1c407b0f5b2805, 0xbe7bce4a600d3ea4),
(0xbaa57d1e1e88c9ca, 0x3e0a6ee796b871b6),
(0x3a3b6e7030a77899, 0xbd92393d82c6b2e4),
(0x397fcfedacb03781, 0x3d131085da82054c),
(0xb8e45f51f6118e46, 0xbc8f4ed4b492ebcc),
(0xb89bd46046c3c8de, 0x3c04b7ac8a1b15d0),
(0x37d1a206fb205e32, 0xbb769201941d0d49),
(0x3782f38acbf23993, 0x3ae4987e587ab039),
(0x36b691bdabf5672b, 0xba4ff1953e0a7c5b),
(0x3636e1c8cd260e18, 0x39b55031dc5e1967),


In [177]:
# log coeffs

print_double_double("", RealField(107)('0.33333333333333333333333333701941430830813680984317'))
print_double_double("", RealField(107)('-0.25000000000000000000000000927469621095374516819563'))
print_double_double("", RealField(107)('0.19999999999999999999736148023407947320974191411113'))
print_double_double("", RealField(107)('-0.166666666666666666661740817355617089084687157682226'))
print_double_double("", RealField(107)('0.142857142857143514208845068731577058039341433616166'))
print_double_double("", RealField(107)('-0.125000000000000975543372027000681903260219080482025'))
print_double_double("", RealField(107)('0.111111111036972084401481914727934733718551705198942'))
print_double_double("", RealField(107)('-9.9999999908474789172229905677506311880264028108553e-2'))
print_double_double("", RealField(107)('9.0912976618933058600055577564907185974457745483036e-2'))
print_double_double("", RealField(107)('-8.333744694635135507057149825265760651408947697174e-2'))
print_double_double("", RealField(107)('7.6927691777549212280402265589168123489454885488853e-2'))
print_double_double("", RealField(107)('-7.1432719425306658106725949877427166499943643429565e-2'))

(0x3c755555556795ff, 0x3fd5555555555555),
(0xba86f68980000000, 0xbfd0000000000000),
(0xbc699b285263b391, 0x3fc999999999999a),
(0xbc65526cf5c49dc3, 0xbfc5555555555555),
(0xbc34fc756f340748, 0x3fc24924924924aa),
(0xbc52e654a63b293e, 0xbfc0000000000023),
(0xbc5c73c13a9c2171, 0x3fbc71c71c2042d5),
(0xbc3d2af5e7ee68d8, 0xbfb999999934f78b),
(0xbc590d76077808da, 0x3fb74612a55c3e99),
(0xbc3447161ca8047c, 0xbfb5559a592aadc7),
(0x3c18760148059540, 0x3fb3b18880576230),
(0xbc3b2424b7665f98, 0xbfb2496a29cbc904),


In [3]:
R120 = RealField(120)

zeros = []

mp.prec = 150

step = mpf("0.1")
epsilon = mpf("1e-35")
x = mpf("1.25")

previous_zero = R120(0)
y0_zeros = []

while x < mpf("76.0"):
    f1 = bessely(0, x)
    f2 = bessely(0, x + step)
    if f1 * f2 < 0:
        zero = findroot(lambda t: bessely(0, t), (x, x + step), solver='secant', tol=mp.mpf("1e-41"))
        previous_zero = zero
        y0_zeros.append(zero)
    if previous_zero is not None and abs(x - mpf(f'{round(x)}')) < epsilon:
        zeros.append(previous_zero)
    x += step

y0_extrema = []

x = mpf("1.25")
while x < mpf("76.0"):
    d1 = mp.diff(lambda t: bessely(0, t), x)
    d2 = mp.diff(lambda t: bessely(0, t), x + step)
    if d1 * d2 < 0:
        extremum = findroot(lambda t: mp.diff(lambda u: bessely(0, u), t), (x, x + step), solver='secant', tol=mp.mpf("1e-41"))
        y0_extrema.append(extremum)
    x += step

# Print results
for i, z in enumerate(y0_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of Y0(x):")
for e in y0_extrema:
    print(f"nExtrema: {e}")

y0_zeros.extend(y0_extrema)

y0_zeros = sorted(y0_zeros)

# Print results
for i, z in enumerate(y0_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

# print("pub(crate) static Y0_ZEROS: [(u64, u64); 48] = [")
# print(f"(0x0, 0x0),")
# for z in y0_zeros:
#     k = split_double_double(z)
#     hi = double_to_hex(k[1])
#     lo = double_to_hex(k[0])
#     print(f"({lo}, {hi}),")
    
# print("];")

print("pub(crate) static Y0_ZEROS_RATIONAL128: [DyadicFloat128; 48] = [")
print(f"DyadicFloat128 {{ sign: DyadicSign::Pos, exponent: 0, mantissa: 0x0u128, }},")
for z in y0_zeros:
    print_dyadic(z)
    
print("];")

Zero 1: x ≈ 3.9576784193148578683756771869174012814186038
Zero 2: x ≈ 7.0860510603017726976236245968203524689715104
Zero 3: x ≈ 10.22234504349641701899204227634218712599406
Zero 4: x ≈ 13.361097473872763478267694585713786426579135
Zero 5: x ≈ 16.500922441528090753421143666489774115751333
Zero 6: x ≈ 19.641309700887939773776045472285980025441517
Zero 7: x ≈ 22.782028047291559316932081968396516662816306
Zero 8: x ≈ 25.922957653180922706872191146269373317052509
Zero 9: x ≈ 29.064030252728398055304718405181344393605068
Zero 10: x ≈ 32.205204116493280650953559661516194096980979
Zero 11: x ≈ 35.346452305214320508799868706358214091282447
Zero 12: x ≈ 38.487756653081537176464556122066562157372007
Zero 13: x ≈ 41.629104466213807963834028403266947835118214
Zero 14: x ≈ 44.770486607221993156366801616102992110196472
Zero 15: x ≈ 47.911896331516480342150305098366009288219297
Zero 16: x ≈ 51.053328552362362084622730461625767074078664
Zero 17: x ≈ 54.19477936108705429869041017135112434866769
Zero 18:

In [113]:
# Taylor series for f32
mp.prec = 60
terms = 28
print(f"pub(crate) static Y0F_COEFFS: [[u64; {terms}]; {len(j0_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly, n):
    print("[")
    for i in range(0, n):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(y0_zeros)):
    k_range = y0_zeros[i]
    range_diff = k_range - prev_zero

    x0 = mp.mpf(k_range)
    from mpmath import mp, bessely, taylor
    poly = taylor(lambda val: bessely(0, val), x0, terms)
    # print(poly)
    print_taylor_coeffsf(poly, terms)
    prev_zero = y0_zeros[i]

print("];")

pub(crate) static Y0F_COEFFS: [[u64; 41]; 47] = [
[mpf('0.52078641240226751110715073692607'), mpf('0.0'), mpf('-0.26039320620113375555357536846304'), mpf('0.0395048485830322028364602785614'), mpf('0.0082143493513216024279099048784476'), mpf('0.00095956233402108751324624666282015'), mpf('-0.0012370922221179114212488122443617'), mpf('0.00037074881260917165021899792834161'), mpf('-0.00013335662500684747464830109816763'), mpf('0.00005662236715064509475652994381275'), mpf('-0.00002358699837551760903508966274673'), mpf('0.0000097964034682169090390241456552774'), mpf('-0.0000041036905126218338307613733655675'), mpf('0.0000017300870834862234846237408636078'), mpf('-0.00000073313228183263465617037950680149'), mpf('0.00000031208598237787635042276363091502'), mpf('-0.00000013339044438352910154921218452208'), mpf('0.000000057219051205514850171026599282661'), mpf('-0.00000002462396619372747864989152467005'), mpf('0.000000010627628156707689361120006498825'), mpf('-0.000000004598907005770772840957337

In [17]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = 1.35 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.05
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(y0_zeros)
print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 500;
bessel_y0 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_y0(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RR(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

degree = 19

print(f"pub(crate) static Y0F_COEFFS: [[u64;{degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print("[")
    for c in coeffs:
        print(double_to_hex(c) + ",")
    print("],")
print("];")

[(-0.84714, 0.93027, 2.1971413260310170351490335626990), (-0.93027, 0.78600, 3.9576784193148578683756771869174), (-0.78600, 0.87819, 5.4296810407941351327720051908526), (-0.87819, 0.80498, 7.0860510603017726976236245968204), (-0.80498, 0.86317, 8.5960058683311689264296061801640), (-0.86317, 0.81340, 10.222345043496417018992042276342), (-0.81340, 0.85597, 11.749154830839881243399421939922), (-0.85597, 0.81817, 13.361097473872763478267694585714), (-0.81817, 0.85174, 14.897442128336725378844819156430), (-0.85174, 0.82124, 16.500922441528090753421143666490), (-0.82124, 0.84895, 18.043402276727855564304555507890), (-0.84895, 0.82338, 19.641309700887939773776045472286), (-0.82338, 0.84698, 21.188068934142213016142481528685), (-0.84698, 0.82496, 22.782028047291559316932081968397), (-0.82496, 0.84551, 24.331942571356912035992944051850), (-0.84551, 0.82617, 25.922957653180922706872191146269), (-0.82617, 0.84437, 27.475294980449223512212285525411), (-0.84437, 0.82713, 29.064030252728398055304718

In [5]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = 1.35 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.03 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.03 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.03 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.03
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.03  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(y0_zeros)
# print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 200;
bessel_y0 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_y0(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_remez_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print_double_double("", coeff)
    print("],")
    
def print_remez_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print_double_double("", coeff)
    print("],")

degree = 27

print(f"pub(crate) static Y0_COEFFS: [[(u64, u64); {degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print_remez_coeffs(coeffs)
print("];")

pub(crate) static Y0_COEFFS_RATIONAL128: [[DyadicFloat128; 30]; 47] = [
[
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -128,
    mantissa: 0x85524221_780a56b6_c9205cc1_b78aab9f_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -196,
    mantissa: 0xabcd8153_5673ae25_0ec01d46_2f5d77aa_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -129,
    mantissa: 0x85524221_780a56b5_05c81cec_c8e94874_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -132,
    mantissa: 0xa1cfd60b_292fb294_419fdc54_a2f3eb1d_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -134,
    mantissa: 0x86957a74_991c17e1_c90d6924_dfc2aec1_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -138,
    mantissa: 0xfb8b235f_54015d66_1771ef94_462936bd_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -137,
    mantissa: 0xa225ed45_b30b8605_21ec3cfe_eb985b68_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exp

KeyboardInterrupt: 

In [10]:
# # Taylor series for f64
# mp.prec = 110
# terms = 50
# print(f"pub(crate) static Y0_COEFFS: [[(u64, u64); {terms}]; {len(y0_zeros)}] = [")

# def get_constant_term(poly, y):
#     for term in poly.operands():
#         if term.is_constant():
#             return term

# def print_taylor_coeffs(poly, n):
#     print("[")
#     for i in range(0, n):
#         coeff = poly[i]
#         print_double_double("", RealField(300)(coeff))
#         # print(f"{double_to_hex(coeff)},")
#     print("],")

# prev_zero = 0

# for i in range(0, len(y0_zeros)):
#     k_range = y0_zeros[i]
#     range_diff = k_range - prev_zero

#     x0 = mp.mpf(k_range)
#     from mpmath import mp, bessely, taylor, chebyfit
#     poly = taylor(lambda val: bessely(0, val), x0, terms)
#     print_taylor_coeffs(poly, terms)
#     prev_zero = y0_zeros[i]

# print("];")

# Taylor series for f64
mp.prec = 140
terms = 28
print(f"pub(crate) static Y0_COEFFS_RATIONAL128: [[DyadicFloat128; {terms}]; {len(y0_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_dyadic_cef(poly, n):
    print("[")
    for i in range(0, n):
        coeff = poly[i]
        print_dyadic(coeff)
        # print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(y0_zeros)):
    k_range = y0_zeros[i]
    range_diff = k_range - prev_zero

    x0 = mp.mpf(k_range)
    from mpmath import mp, bessely, taylor, chebyfit
    poly = taylor(lambda val: bessely(0, val), x0, terms)
    print_taylor_dyadic_cef(poly, terms)
    prev_zero = y0_zeros[i]

print("];")

pub(crate) static Y0_COEFFS_RATIONAL128: [[DyadicFloat128; 28]; 47] = [
];
